In [ ]:
!pip install opencv-python-headless deepface moviepy

import cv2
from deepface import DeepFace
import moviepy.editor as mp
from google.colab import drive

drive.mount('/content/drive')
video_path = "/content/drive/My Drive/face_videos/PSV.mp4"

In [ ]:
# Open the video file using OpenCV
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

# Get video details
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Define the output video path on Google Drive
output_video_path = "/content/drive/My Drive/face_videos/PSV_output_video.mp4"

# Create a VideoWriter object to save the output video
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Define the text color (Lila color in BGR)
text_color = (255, 0, 255)  # Lilac

# Define the rectangle color (Green in BGR)
rectangle_color = (0, 255, 0)  # Green

# Process the video frame by frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Detect faces and analyze each face in the frame using DeepFace
    try:
        # Analyze the frame for face detection, emotion, and gender
        results = DeepFace.analyze(frame, actions=['emotion', 'gender'], enforce_detection=False)

        # Process each detected face in the frame
        for result in results:
            # Get face bounding box coordinates
            x, y, w, h = result['region']['x'], result['region']['y'], result['region']['w'], result['region']['h']

            # Draw a rectangle around the face (Green color)
            cv2.rectangle(frame, (x, y), (x + w, y + h), rectangle_color, 2)

            # Get the dominant emotion and gender probabilities
            dominant_emotion = result['dominant_emotion']
            gender_probabilities = result['gender']

            # Determine the gender with the highest probability
            predicted_gender = max(gender_probabilities, key=gender_probabilities.get)
            predicted_gender_prob = gender_probabilities[predicted_gender]

            # Annotate the frame with the detected emotion and the dominant gender with its probability
            text = f"{dominant_emotion}, {predicted_gender} ({predicted_gender_prob*100:.2f}%)"
            cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, text_color, 2, cv2.LINE_AA)

    except Exception as e:
        print(f"Error processing frame: {e}")
        continue

    # Write the annotated frame to the output video
    out.write(frame)

# Release the video capture and writer objects
cap.release()
out.release()

In [ ]:
# Convert the output to a format that can be viewed in Colab // does work for some videos but not all :(
clip = mp.VideoFileClip(output_video_path)
clip.ipython_display(width=640)